### Embedding Queries

In [42]:
from fastembed import TextEmbedding
import numpy as np
from typing import List

In [30]:
embedding_model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

In [ ]:
documents: list[str] = [
    'I just discovered the course. Can I join now?'
]
embeddings_generator = embedding_model.embed(documents)  # reminder this is a generator
embeddings_list = list(embedding_model.embed(documents))
  # you can also convert the generator to a list, and that to a numpy array
len(embeddings_list[0]) # Vector of 512 dimensions

512

In [32]:
np.min(embeddings_list[0])

np.float64(-0.11726373885183883)

### Cosine Similarity

In [33]:
import numpy as np

np.linalg.norm(embeddings_list[0])

np.float64(1.0)

In [34]:
embeddings_list[0].dot(embeddings_list[0])

np.float64(1.0000000000000002)

In [35]:
documents: list[str] = [
    'Can I still join the course after the start date?'
]
embeddings_generator = embedding_model.embed(documents)  # reminder this is a generator
embeddings_list_second_q = list(embedding_model.embed(documents))
  # you can also convert the generator to a list, and that to a numpy array
len(embeddings_list_second_q[0]) # Vector of 512 dimensions

512

In [37]:
embeddings_list_second_q[0].dot(embeddings_list[0])

np.float64(0.9008528895674548)

### Ranking by Cosine

In [39]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [47]:
doc_text_lst = [doc['text'] for doc in documents]

In [50]:
def embed_text(document_list: List[str], embedding_model: TextEmbedding) -> List[np.ndarray]:
    embeddings_list = list(embedding_model.embed(document_list))

    return embeddings_list

In [53]:
doc_text_embeddings = embed_text(document_list=doc_text_lst, embedding_model=embedding_model)

In [62]:
cosine_sim_array = np.array(doc_text_embeddings).dot(query_embed)

In [71]:
np.argmax(cosine_sim_array),  np.max(cosine_sim_array)

(np.int64(1), np.float64(0.8182378150042889))

### Ranking by Cosine V2

In [ ]:
full_text_lst = [doc['question'] + ' ' + doc['text'] for doc in documents]

In [66]:
full_text_embeddings = embed_text(document_list=full_text_lst, embedding_model=embedding_model)

In [67]:
full_text_cosine_sim_array = np.array(full_text_embeddings).dot(query_embed)

The full text has more keywords that align with the query than just the text field. By adding the question into the embedding, the first document becomes much more similar to the query

In [70]:
np.argmax(full_text_cosine_sim_array), np.max(full_text_cosine_sim_array)

(np.int64(0), np.float64(0.8514543236908068))

### Embedding Model Selection

In [75]:
min_dim = 0

for idx, model in enumerate(TextEmbedding.list_supported_models()):

    if idx == 0:
        min_dim = model['dim']
        continue

    if model['dim'] < min_dim:
        min_dim = model['dim']


In [76]:
print(min_dim)

384


#### Indexing with Qdrant

In [77]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [78]:
from qdrant_client import QdrantClient, models

qdrant_client = QdrantClient("http://localhost:6333")

In [79]:
model_handle = "BAAI/bge-small-en"

In [88]:
qdrant_client.create_collection(
    collection_name = "mlzoopcamp-rag",
    vectors_config=models.VectorParams(
        size=384,
        distance=models.Distance.COSINE
    )
    
)

True

In [89]:
import uuid 

qdrant_client.upsert(
    collection_name = "mlzoopcamp-rag",
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector = models.Document(
                    text=doc['question'] + ' ' + doc['text'],
                    model=model_handle,
                ),
            payload={
                "text": doc['question'] + ' ' + doc['text'],
                "section": doc["section"],
                "course": course["course"]
            }
        )
       
        for doc in documents
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [91]:
def search(query, limit=3):

    results = qdrant_client.query_points(
        collection_name="mlzoopcamp-rag",
        query=models.Document( # embed the query text locally with jinaai
            text=query,
            model=model_handle

        ),
        limit=limit,  # top closest matches
        with_payload=True # to get metadata in the results

    )

    return results

In [93]:
results = search("I just discovered the course. Can I join now?")

In [110]:
results.points[0].score

0.8703172